In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder



2023-07-20 18:06:58.563511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 18:06:59.302156: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/panda/test_ws/devel/lib:/home/panda/ws_moveit/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2023-07-20 18:06:59.302229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ho

In [2]:
data_dir = '/home/panda/model_data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['None','push', 'pull', 'up', 'down', 'stop','rotate_clockwise','rotate_counterclockwise','rotate_up','rotate_down','grasp','fast','slow']
# Iterate over each subdirectory (class folder) in the data directory
for i,class_name in enumerate    (class_names):
    
    class_path = os.path.join(data_dir, class_names[i])
    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            # count+=1
            if file_name.endswith('.npy'):
                  
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                # Check if the shape of the recorded array matches the expected shape
                expected_shape = (210, )  # Adjust the shape according to your data
                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    count+=1
                    recorded_array=[]
                else:
                    continue
    print(f'data points in class {class_name} are {count}') 
    count=0         
# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Step 2: Preprocess the data

# Perform any required preprocessing steps, such as scaling or normalization
# Here, we will use StandardScaler to scale the data
print(data.shape)
print (labels.shape)
print(len(class_names))
print(count)

data points in class None are 2500
data points in class push are 2000
data points in class pull are 2000
data points in class up are 2000
data points in class down are 2000
data points in class stop are 4000
data points in class rotate_clockwise are 2000
data points in class rotate_counterclockwise are 2000
data points in class rotate_up are 1994
data points in class rotate_down are 2000
data points in class grasp are 2000
data points in class fast are 2000
data points in class slow are 2000
(28494, 210)
(28494,)
13
0


In [81]:
data_dir = '/home/rehan/catkin_ws/src/panda_research/hand_classifier/data/cnn_data'

# Step 1: Load and preprocess the data

# Initialize empty lists to store the data and labels
data = []
labels = []
recorded_array=[]
count=0
class_names = ['push', 'pull', 'up', 'down', 'stop','rotate_cw','rotate_ccw']
class_counts = []  # Initialize a list to store the counts of valid samples per class

# Iterate over each subdirectory (class folder) in the data directory
for i, class_name in enumerate(class_names):
    class_path = os.path.join(data_dir, class_name)
    class_count = 0  # Initialize the count for the current class

    if os.path.isdir(class_path):
        # Iterate over each sample file in the class folder
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)

            if file_name.endswith('.npy'):
                # Load the recorded array from the file
                recorded_array = np.load(file_path)
                expected_shape = (210,)  # Adjust the shape according to your data

                if recorded_array.shape == expected_shape:
                    # Append the recorded array to the data list
                    data.append(recorded_array.flatten())
                    # Append the corresponding label to the labels list
                    label = class_names.index(class_name)
                    labels.append(label)
                    class_count += 1  # Increment the count for the current class

    class_counts.append(class_count)  # Store the count for the current class

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Print the counts for each class
for i, class_name in enumerate(class_names):
    print(f"{class_name}: {class_counts[i]}")

push: 2000
pull: 2000
up: 2000
down: 2000
stop: 2000
rotate_cw: 2000
rotate_ccw: 2000


In [3]:
# Split the data and labels into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
normalizer = MinMaxScaler(feature_range=(0, 1))

data_scaled = scaler.fit_transform(data)
data_normalized = normalizer.fit_transform(data_scaled)

train_data, val_data, train_labels, val_labels = train_test_split(data_normalized, labels, test_size=0.2, random_state=42)

print(train_data.shape)
print(val_data.shape)
print(train_labels.shape)
print(val_labels.shape)

(22795, 210)
(5699, 210)
(22795,)
(5699,)


In [5]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(210,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))


2023-07-20 18:08:07.635365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 18:08:08.747293: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6874 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:65:00.0, compute capability: 6.1


In [6]:

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [7]:
# Train the model
model.fit(train_data, train_labels, batch_size=32, epochs=100, validation_data=(val_data, val_labels))

Epoch 1/100
713/713 [==============================] - 5s 6ms/step - loss: 1.6179 - accuracy: 0.4522 - val_loss: 0.3991 - val_accuracy: 0.9437
Epoch 2/100
713/713 [==============================] - 4s 6ms/step - loss: 0.7421 - accuracy: 0.7503 - val_loss: 0.1595 - val_accuracy: 0.9667
Epoch 3/100
713/713 [==============================] - 4s 6ms/step - loss: 0.5680 - accuracy: 0.8037 - val_loss: 0.1064 - val_accuracy: 0.9777
Epoch 4/100
713/713 [==============================] - 4s 6ms/step - loss: 0.4795 - accuracy: 0.8312 - val_loss: 0.0758 - val_accuracy: 0.9881
Epoch 5/100
713/713 [==============================] - 4s 6ms/step - loss: 0.4335 - accuracy: 0.8484 - val_loss: 0.0511 - val_accuracy: 0.9923
Epoch 6/100
713/713 [==============================] - 4s 6ms/step - loss: 0.4007 - accuracy: 0.8593 - val_loss: 0.0488 - val_accuracy: 0.9909
Epoch 7/100
713/713 [==============================] - 4s 6ms/step - loss: 0.3682 - accuracy: 0.8725 - val_loss: 0.0512 - val_accuracy: 0.9884

In [8]:
# Get the 9predicted labels from the model
predicted_labels = model.predict(val_data)
predictions = np.around(model.predict(val_data))
# print(predictions)
# Decode the predicted labels
print((predicted_labels.shape))
for i,prediction in enumerate (predictions):
    # print(np.argmax(predictions[i][0]))
    label=np.argmax(predictions)
    print(class_names[label])
# print(prediction_labels[0][np.argmax(prediction)])
print(class_names)

179/179 [==============================] - 0s 2ms/step
(5699, 13)
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_down
rotate_dow

In [21]:
import pickle
# Save the model
model.save('/home/panda/model_data/model4/keras_model.h5')

# Save the labels to a text file
with open('/home/panda/model_data/model4/labels.txt', 'w') as f:
    for label in class_names:
        f.write(label + '\n')
# Saving the scaler and normalizer objects
with open('/home/panda/model_data/model4/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

with open('/home/panda/model_data/model4/normalizer.pkl', 'wb') as normalizer_file:
    pickle.dump(normalizer, normalizer_file)